In [16]:
import numpy as np
import cv2

In [17]:
cap = cv2.VideoCapture('/Users/danielrosenthal/Documents/EGR_314/Matt_side3_720.mov')
if cap.isOpened() == False:
    print("Cannot open video file.")

#object detection
object_detector = cv2.createBackgroundSubtractorMOG2()

In [18]:
while True:
    ret, frame = cap.read()

    mask = object_detector.apply(frame)

    
    cv2.imshow("Frame",frame)
    cv2.imshow("Mask", mask)

    key = cv2.waitKey(120)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()